# Projeto: Detecção de Queimadas com Machine Learning
Este projeto tem como objetivo detectar áreas queimadas utilizando aprendizado de máquina com dados geoespaciais e espectrais.


In [ ]:
%%capture
!pip install -q --upgrade pip
!pip install -q segmentation-models-pytorch torchmetrics albumentations kagglehub torchvision pillow matplotlib scikit-learn tensorboard tqdm


In [2]:
import os

# Caminho raiz do dataset principal
dataset_path = "./dataset_kaggle/dataset"

def get_paths(dataset_path):
    images_paths = []
    mask_paths = []

    for dirname, _, filenames in os.walk(dataset_path):
        for filename in filenames:
            full_path = os.path.join(dirname, filename)
            if "mask" in dirname.lower():   # se a pasta tem 'mask' no nome
                mask_paths.append(full_path)
            else:
                images_paths.append(full_path)

    # Ordena para manter consistência
    images_paths = sorted(images_paths)
    mask_paths   = sorted(mask_paths)
    return images_paths, mask_paths

images_paths, mask_paths = get_paths(dataset_path)
print(f"Total: {len(images_paths)} imagens | {len(mask_paths)} máscaras")

Total: 1890 imagens | 945 máscaras


In [3]:
# Mostrar alguns caminhos pra validar se carregou certo
print("Exemplos de imagens:")
print(images_paths[:5])

print("\nExemplos de máscaras:")
print(mask_paths[:5])

Exemplos de imagens:
['./dataset_kaggle/dataset\\t1\\recorte_1.tif', './dataset_kaggle/dataset\\t1\\recorte_10.tif', './dataset_kaggle/dataset\\t1\\recorte_100.tif', './dataset_kaggle/dataset\\t1\\recorte_101.tif', './dataset_kaggle/dataset\\t1\\recorte_102.tif']

Exemplos de máscaras:
['./dataset_kaggle/dataset\\mask\\recorte_1.tif', './dataset_kaggle/dataset\\mask\\recorte_10.tif', './dataset_kaggle/dataset\\mask\\recorte_100.tif', './dataset_kaggle/dataset\\mask\\recorte_101.tif', './dataset_kaggle/dataset\\mask\\recorte_102.tif']


In [4]:
from PIL import Image
import matplotlib.pyplot as plt

# Visualizar a primeira imagem e a respectiva máscara
img = Image.open(images_paths[0])
mask = Image.open(mask_paths[0])

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(img)
plt.title("Imagem (Tensor)")

plt.subplot(1,2,2)
plt.imshow(mask, cmap="gray")
plt.title("Máscara")

plt.show()

UnidentifiedImageError: cannot identify image file './dataset_kaggle/dataset\\t1\\recorte_1.tif'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, labels, test_size=0.2, random_state=42)

In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

In [ ]:
y_pred_proba = clf.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)

fpr, tpr, _ = roc_curve(y_test, y_pred_proba)

plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {roc_auc:.2f})")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.grid(True)
plt.show()